# scrape multiple websites

In [1]:
from __future__ import annotations

# DATE PARAMETERS
from datetime import date, datetime, timezone, timedelta

START_DATE = date(2020, 1, 1)   # inclusive
END_DATE   = date(2025, 10, 31)  # inclusive

# OUTPUT FILE
OUTPUT_XLSX_ALL = "Output/CNA/cna_articles_2020_Jan_to_2025_Oct.xlsx"


In [2]:
# WEBSITES TO SCRAPE FROM
TOPIC_JOBS = [
    {
        "name": "housing-and-development-board",
        "topic_url": "https://www.channelnewsasia.com/topic/housing-and-development-board"
    },
    {
        "name": "housing",
        "topic_url": "https://www.channelnewsasia.com/topic/housing"
    },
    {
        "name": "private-housing",
        "topic_url": "https://www.channelnewsasia.com/topic/private-housing"
    },
    {
        "name": "property-market",
        "topic_url": "https://www.channelnewsasia.com/topic/property-market"
    },
    {
        "name": "hdb-flats",
        "topic_url": "https://www.channelnewsasia.com/topic/hdb-flats"
    },
    {
        "name": "hdb",
        "topic_url": "https://www.channelnewsasia.com/topic/hdb"
    },
    {
        "name": "bto",
        "topic_url": "https://www.channelnewsasia.com/topic/bto"
    },
    {
        "name": "hdb-flat",
        "topic_url": "https://www.channelnewsasia.com/topic/hdb-flats" # typo in this website, should be 'hdb-flat', so have to rescrape in the codes below
    },
    {
        "name": "public-housing",
        "topic_url": "https://www.channelnewsasia.com/topic/public-housing"
    },
    {
        "name": "property",
        "topic_url": "https://www.channelnewsasia.com/topic/property"
    },
    {
        "name": "private-home-sales",
        "topic_url": "https://www.channelnewsasia.com/topic/private-home-sales"
    },
    {
        "name": "home-ownership",
        "topic_url": "https://www.channelnewsasia.com/topic/home-ownership"
    },
    {
        "name": "property-tax",
        "topic_url": "https://www.channelnewsasia.com/topic/property-tax"
    }
]

In [3]:
from __future__ import annotations

import json
import os
import re
import time
from datetime import date, datetime, timezone, timedelta
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook

# --------------------------
# GLOBAL CONFIG
# --------------------------

REQUEST_TIMEOUT = 20
SLEEP_BETWEEN_LISTING = 1.0   # seconds when crawling listing
SLEEP_BETWEEN_ARTICLES = 0.6  # seconds when scraping article pages

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}

SGT = timezone(timedelta(hours=8))
ISO_DT_RE = re.compile(r"^\d{4}-\d{2}-\d{2}")

# For article-body cleaning
STOP_MARKER = "sign up for our newsletters"
_BOILERPLATE_PATTERNS = [
    r"Get CNA updates on WhatsApp",
    r"Subscribe to our newsletter",
    r"Related:?$",
    r"Recommended(?: for you)?:?$",
    r"READ:?$",
    r"READ ALSO:?$",
    r"Follow us on",
    r"Read more:?$",
    r"Get the CNA app",
    r"Get WhatsApp alerts",
    r"Also worth reading",
]
_BOILERPLATE_RE = re.compile("|".join(_BOILERPLATE_PATTERNS), re.IGNORECASE)

_CONTENT_CLASS_RE = re.compile(
    r"(article|story|post).*(content|body)|"
    r"(content|text|rich)(-|_)?(body|area|container)",
    re.IGNORECASE
)

# --------------------------
# CHECKPOINT CONFIG
# --------------------------

CHECKPOINT_DIR = "checkpoints_cna"
ARTICLE_CACHE_CKPT = os.path.join(CHECKPOINT_DIR, "articles_cache.json")
ALL_ROWS_CKPT = os.path.join(CHECKPOINT_DIR, "all_rows.json")
# per-topic URLs: urls_<topic_name>.json in CHECKPOINT_DIR


# --------------------------
# UTILS
# --------------------------
def ensure_dirs(path: str):
    d = os.path.dirname(path)
    if d:
        os.makedirs(d, exist_ok=True)


def load_json(path: str, default):
    if not os.path.exists(path):
        return default
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception as e:
        print(f"[WARN] Failed to load JSON checkpoint {path}: {e}")
        return default


def save_json(path: str, obj):
    ensure_dirs(path)
    try:
        with open(path, "w", encoding="utf-8") as f:
            json.dump(obj, f, ensure_ascii=False, indent=2)
    except Exception as e:
        print(f"[WARN] Failed to save JSON checkpoint {path}: {e}")


def topic_urls_checkpoint_path(topic_name: str) -> str:
    safe_name = re.sub(r"[^a-zA-Z0-9_\-]+", "_", topic_name)
    return os.path.join(CHECKPOINT_DIR, f"urls_{safe_name}.json")


# --------------------------
# SHARED HELPERS
# --------------------------
def fetch(url: str) -> str | None:
    try:
        r = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT)
        r.encoding = "utf-8"
        r.raise_for_status()
        return r.text
    except Exception as e:
        print(f"[WARN] GET failed: {url} | {e}")
        return None


def is_article_url(url: str) -> bool:
    """
    CNA-specific heuristic for article URLs.
    """
    u = urlparse(url)
    if u.netloc not in {"www.channelnewsasia.com", "channelnewsasia.com"}:
        return False

    path = u.path.lower()

    # Exclude obvious non-article paths
    if path.startswith("/watch") or "/watch/" in path:
        return False
    if "/podcast" in path or "/podcasts" in path or path.startswith("/listen"):
        return False
    if path.startswith("/profile") or path.startswith("/rss") or path.startswith("/weather"):
        return False
    if path.startswith("/topic/") or path.startswith("/tag/") or path.startswith("/category/"):
        return False
    if path == "/":
        return False

    # CNA articles usually end with an ID, e.g. ...-5422501
    if not any(ch.isdigit() for ch in path):
        return False

    return True


def within_range(dt: datetime) -> bool:
    d = dt.date()
    return START_DATE <= d <= END_DATE


def date_is_before_start(dt: datetime) -> bool:
    return dt.date() < START_DATE


def parse_publish_datetime_sgt(html: str) -> datetime | None:
    """
    Parse publish datetime from article HTML and return timezone-aware datetime in SGT.
    """
    soup = BeautifulSoup(html, "html.parser")

    # 1) JSON-LD
    for script in soup.find_all("script", type="application/ld+json"):
        try:
            data = json.loads(script.string or "")
        except Exception:
            continue

        candidates = data if isinstance(data, list) else [data]
        for obj in candidates:
            if not isinstance(obj, dict):
                continue
            typ = obj.get("@type") or obj.get("@context")
            if (isinstance(typ, str) and "Article" in typ) or obj.get("headline") or obj.get("datePublished"):
                dt = obj.get("datePublished") or obj.get("dateModified")
                if isinstance(dt, str) and ISO_DT_RE.match(dt):
                    try:
                        dt_parsed = datetime.fromisoformat(dt.replace("Z", "+00:00"))
                        return dt_parsed.astimezone(SGT)
                    except Exception:
                        pass

    # 2) Meta tags
    meta_names = [
        ("meta", {"property": "article:published_time"}),
        ("meta", {"name": "pubdate"}),
        ("meta", {"name": "publish-date"}),
        ("meta", {"name": "date"}),
        ("meta", {"itemprop": "datePublished"}),
        ("meta", {"property": "og:pubdate"}),
        ("meta", {"name": "parsely-pub-date"}),
        ("meta", {"name": "cXenseParse:recs:publishtime"}),
    ]
    for tag, attrs in meta_names:
        el = soup.find(tag, attrs=attrs)
        if el and el.get("content"):
            dt = el["content"].strip()
            if ISO_DT_RE.match(dt):
                try:
                    dt_parsed = datetime.fromisoformat(dt.replace("Z", "+00:00"))
                    return dt_parsed.astimezone(SGT)
                except Exception:
                    pass

    # 3) <time> tags
    for t in soup.find_all("time"):
        dt = (t.get("datetime") or t.get("content") or "").strip()
        if ISO_DT_RE.match(dt):
            try:
                dt_parsed = datetime.fromisoformat(dt.replace("Z", "+00:00"))
                return dt_parsed.astimezone(SGT)
            except Exception:
                continue

    return None


# --------------------------
# TOPIC LISTING → URL LIST
# --------------------------
def extract_listing_links(topic_html: str, topic_url: str) -> list[str]:
    """
    Extract candidate article URLs from a CNA topic page.
    Prefer `data-link` attributes, fallback to <a href>.
    """
    soup = BeautifulSoup(topic_html, "html.parser")
    hrefs = set()

    # 1) data-link
    for el in soup.find_all(attrs={"data-link": True}):
        data_link = el.get("data-link")
        if not data_link:
            continue
        if data_link.startswith("#"):
            continue
        abs_url = urljoin(topic_url, data_link)
        if is_article_url(abs_url):
            hrefs.add(abs_url)

    # 2) fallback: <a href="">
    if not hrefs:
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if href.startswith("#"):
                continue
            abs_url = urljoin(topic_url, href)
            if is_article_url(abs_url):
                hrefs.add(abs_url)

    return sorted(hrefs)


def collect_urls_for_topic(topic_name: str, topic_url: str) -> list[str]:
    """
    Crawl a CNA topic listing and return article URLs within date range.
    Crawl until we run out of pages or mostly hit items older than START_DATE.
    Uses per-topic checkpoint to avoid losing progress.
    """
    print(f"\n=== Topic: {topic_name} | {topic_url} ===")
    print(f"Date range: {START_DATE} to {END_DATE}")

    ckpt_path = topic_urls_checkpoint_path(topic_name)
    ckpt_data = load_json(ckpt_path, default=None)

    if ckpt_data:
        urls_in_range = ckpt_data.get("urls_in_range", [])
        last_page = ckpt_data.get("last_page", -1)
        reached_older_than_start = ckpt_data.get("reached_older_than_start", False)
        print(f"[CHECKPOINT] Loaded {len(urls_in_range)} URLs from topic checkpoint (last_page={last_page}, reached_older={reached_older_than_start}).")
        if reached_older_than_start:
            # Already fully crawled based on old-date heuristic; just return
            return urls_in_range
        start_page = last_page + 1
    else:
        urls_in_range = []
        reached_older_than_start = False
        start_page = 0

    seen_urls = set(urls_in_range)

    page = start_page
    while True:
        page_url = f"{topic_url}?page={page}"
        print(f"\n[Page {page}] {page_url}")
        html = fetch(page_url)
        if not html:
            print("No HTML returned; stopping.")
            break

        links = extract_listing_links(html, topic_url)
        if not links:
            print("No candidate links found on page; stopping.")
            break

        print(f"Found {len(links)} candidate links; checking dates...")
        page_old_count = 0
        page_in_range = 0

        for article_url in links:
            if article_url in seen_urls:
                continue
            seen_urls.add(article_url)

            time.sleep(SLEEP_BETWEEN_LISTING)
            article_html = fetch(article_url)
            if not article_html:
                print(f"  [SKIP] No HTML for {article_url}")
                continue

            dt = parse_publish_datetime_sgt(article_html)
            if not dt:
                print(f"  [NO DATE] {article_url}")
                continue

            if within_range(dt):
                urls_in_range.append(article_url)
                page_in_range += 1
            elif date_is_before_start(dt):
                page_old_count += 1

        print(f"Kept in range on this page: {page_in_range} | Older-than-start on this page: {page_old_count}")

        # Early stop: this page has nothing in range and lots of too-old articles
        if page_in_range == 0 and page_old_count >= 10:
            reached_older_than_start = True
            print("Reached mostly items older than start date; stopping pagination.")
            # Save final checkpoint and break
            save_json(ckpt_path, {
                "urls_in_range": urls_in_range,
                "last_page": page,
                "reached_older_than_start": True,
            })
            break

        # Save checkpoint after each page
        save_json(ckpt_path, {
            "urls_in_range": urls_in_range,
            "last_page": page,
            "reached_older_than_start": reached_older_than_start,
        })

        # Move to next page
        page += 1

    print(f"-> Topic {topic_name}: collected {len(urls_in_range)} URL(s) in range.")
    if not reached_older_than_start:
        print("  (Note: stopped due to no more pages/links, not old-date heuristic.)")

    # Save final checkpoint state
    save_json(ckpt_path, {
        "urls_in_range": urls_in_range,
        "last_page": page,
        "reached_older_than_start": reached_older_than_start,
    })

    return urls_in_range


# --------------------------
# ARTICLE CONTENT
# --------------------------
def _clean_text(s: str) -> str:
    s = s.strip()
    s = re.sub(r"\xa0", " ", s)
    s = re.sub(r"[ \t]+", " ", s)
    if _BOILERPLATE_RE.search(s):
        return ""
    return s


def _maybe_fix_misencoded(text: str) -> str:
    if "â€" in text or "â€™" in text or "â€“" in text:
        try:
            return text.encode("latin1").decode("utf-8")
        except Exception:
            return text
    return text


def _truncate_after_marker(text: str) -> str:
    if not text:
        return text
    idx = text.lower().find(STOP_MARKER)
    return text[:idx].rstrip() if idx != -1 else text


def extract_article_content(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")

    # 1) JSON-LD articleBody
    try:
        for script in soup.find_all("script", type="application/ld+json"):
            data = json.loads(script.string or "")
            blocks = data if isinstance(data, list) else [data]
            for obj in blocks:
                if isinstance(obj, dict) and obj.get("@type") and "Article" in str(obj.get("@type")):
                    body = obj.get("articleBody")
                    if isinstance(body, str) and len(body.strip()) > 40:
                        text = "\n\n".join(
                            _clean_text(x) for x in re.split(r"\n{2,}", body.strip()) if _clean_text(x)
                        )
                        if text:
                            text = _maybe_fix_misencoded(text)
                            return _truncate_after_marker(text)
    except Exception:
        pass

    def collect_paragraphs(container):
        paras = []
        for p in container.find_all(["p", "h2", "h3"], recursive=True):
            if p.find_parent(["figure", "figcaption", "aside", "blockquote"]):
                continue
            raw = "".join(t for t in p.stripped_strings if isinstance(t, str))
            if STOP_MARKER in raw.lower():
                break
            txt = _clean_text(raw)
            if txt:
                paras.append(txt)
        return paras

    # 2) <article> block
    article_tag = soup.find("article")
    if article_tag:
        for c in article_tag.find_all(attrs={"itemprop": "articleBody"}):
            paras = collect_paragraphs(c)
            if len(" ".join(paras)) > 100:
                return _truncate_after_marker(_maybe_fix_misencoded("\n\n".join(paras)))
        paras = collect_paragraphs(article_tag)
        if len(" ".join(paras)) > 100:
            return _truncate_after_marker(_maybe_fix_misencoded("\n\n".join(paras)))

    # 3) Heuristic containers
    for div in soup.find_all(["div", "section"], class_=True):
        classes = " ".join(div.get("class") or [])
        if _CONTENT_CLASS_RE.search(classes):
            paras = collect_paragraphs(div)
            if len(" ".join(paras)) > 100:
                return _truncate_after_marker(_maybe_fix_misencoded("\n\n".join(paras)))

    # 4) Last resort
    main_like = soup.find("main") or soup.body
    if main_like:
        paras = collect_paragraphs(main_like)
        if len(" ".join(paras)) > 100:
            return _truncate_after_marker(_maybe_fix_misencoded("\n\n".join(paras)))

    return ""


def parse_title(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")
    meta_title = soup.find("meta", property="og:title")
    if meta_title and meta_title.get("content"):
        return meta_title["content"].strip()
    t = soup.find("title")
    return t.text.strip() if t else ""


def scrape_article(url: str) -> dict:
    html = fetch(url)
    if not html:
        raise RuntimeError("Failed to fetch article HTML.")
    title = parse_title(html)
    published_sgt = parse_publish_datetime_sgt(html)
    content = extract_article_content(html)
    return {
        "title": title,
        "url": url,
        "published_date": published_sgt.isoformat() if published_sgt else "",
        "content": content,
        "content_chars": len(content) if content else 0,
    }


# --------------------------
# SAVE EXCEL
# --------------------------
def save_rows_to_excel(rows: list[dict], output_path: str):
    ensure_dirs(output_path)
    wb = Workbook()
    ws = wb.active
    ws.title = "Articles"

    headers = ["website", "title", "url", "published_date", "content", "content_chars"]
    ws.append(headers)

    for r in rows:
        ws.append([r.get(h, "") for h in headers])

    # Optional: auto width
    for column in ws.columns:
        max_length = 0
        col_letter = column[0].column_letter
        for cell in column:
            try:
                val = "" if cell.value is None else str(cell.value)
                if len(val) > max_length:
                    max_length = len(val)
            except Exception:
                pass
        ws.column_dimensions[col_letter].width = min(max_length + 2, 80)

    wb.save(output_path)
    print(f"\n✅ Saved combined Excel file to: {output_path}")


# --------------------------
# MAIN
# --------------------------
def main():
    # Load checkpoints if they exist
    ensure_dirs(CHECKPOINT_DIR)

    article_cache = load_json(ARTICLE_CACHE_CKPT, default={})
    if article_cache:
        print(f"[CHECKPOINT] Loaded {len(article_cache)} articles from article cache.")

    all_rows = load_json(ALL_ROWS_CKPT, default=[])
    if all_rows:
        print(f"[CHECKPOINT] Loaded {len(all_rows)} existing rows from all_rows checkpoint.")

    # Main loop over topics
    for job in TOPIC_JOBS:
        name = job["name"]
        topic_url = job["topic_url"]

        # 1) Get URLs for this topic within date range
        urls = collect_urls_for_topic(name, topic_url)
        print(f"[{name}] Total URLs in range: {len(urls)}")

        # 2) Scrape articles for this topic
        for i, url in enumerate(urls, 1):
            if not is_article_url(url):
                continue
            print(f"[{name}] [{i}/{len(urls)}] Scraping: {url}")
            time.sleep(SLEEP_BETWEEN_ARTICLES)
            try:
                # Only scrape each URL once; reuse cached result for other topics or restarts
                if url not in article_cache:
                    article_data = scrape_article(url)
                    article_cache[url] = article_data
                    # Save article cache checkpoint after each new article
                    save_json(ARTICLE_CACHE_CKPT, article_cache)
                else:
                    article_data = article_cache[url]

                # Make a shallow copy so we can attach topic-specific website info
                row = dict(article_data)
                row["website"] = name
                all_rows.append(row)

                # Save all_rows checkpoint regularly (here: after each article)
                save_json(ALL_ROWS_CKPT, all_rows)

            except Exception as e:
                print(f"[ERROR] {name} | {url} | {e}")

    # Deduplicate by URL only; merge website names if same URL appears in multiple topics
    dedup: dict[str, dict] = {}
    for r in all_rows:
        key = r["url"]
        if key not in dedup:
            dedup[key] = r
        else:
            existing = dedup[key]
            existing_websites = set(existing.get("website", "").split("|")) if existing.get("website") else set()
            if r.get("website") and r["website"] not in existing_websites:
                existing_websites.add(r["website"])
                existing["website"] = "|".join(sorted(w for w in existing_websites if w))

    final_rows = sorted(
        dedup.values(),
        key=lambda r: (r.get("website", ""), r.get("published_date", ""), r["url"])
    )

    # Save final result
    save_rows_to_excel(final_rows, OUTPUT_XLSX_ALL)
    print(f"\nDone. Total unique article rows (by URL): {len(final_rows)}")


if __name__ == "__main__":
    main()



=== Topic: housing-and-development-board | https://www.channelnewsasia.com/topic/housing-and-development-board ===
Date range: 2020-01-01 to 2025-10-31

[Page 0] https://www.channelnewsasia.com/topic/housing-and-development-board?page=0
Found 15 candidate links; checking dates...
Kept in range on this page: 14 | Older-than-start on this page: 0

[Page 1] https://www.channelnewsasia.com/topic/housing-and-development-board?page=1
Found 8 candidate links; checking dates...
Kept in range on this page: 8 | Older-than-start on this page: 0

[Page 2] https://www.channelnewsasia.com/topic/housing-and-development-board?page=2
Found 15 candidate links; checking dates...
Kept in range on this page: 15 | Older-than-start on this page: 0

[Page 3] https://www.channelnewsasia.com/topic/housing-and-development-board?page=3
Found 13 candidate links; checking dates...
Kept in range on this page: 13 | Older-than-start on this page: 0

[Page 4] https://www.channelnewsasia.com/topic/housing-and-developme

# scrape website which was missed out and merge

In [4]:
TOPIC_URL = "https://www.channelnewsasia.com/topic/hdb-flat"
OUTPUT_CSV = "Output/CNA/cna_hdb_flat.csv"

INPUT_CSV = "Output/CNA/cna_hdb_flat.csv"
OUTPUT_XLSX = "Output/CNA/cna_hdb_flat.xlsx"

START_DATE = date(2020, 1, 1)   # inclusive
END_DATE   = date(2025, 10, 31)  # inclusive

from __future__ import annotations

import csv
import json
import re
import time
from datetime import date, datetime, timezone, timedelta
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup

MAX_PAGES = 10  # maximum number of pages to attempt (start at page=0)

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}
REQUEST_TIMEOUT = 20
SLEEP_BETWEEN_REQUESTS = 1.0  # seconds

# --------------------------
# HELPERS
# --------------------------
def is_article_url(url: str) -> bool:
    """
    Heuristic filter for CNA article URLs.
    """
    u = urlparse(url)
    if u.netloc not in {"www.channelnewsasia.com", "channelnewsasia.com"}:
        return False

    path = u.path.lower()

    # Exclude clear non-article sections
    if path.startswith("/watch") or "/watch/" in path:
        return False
    if "/podcast" in path or "/podcasts" in path or path.startswith("/listen"):
        return False
    if path.startswith("/profile") or path.startswith("/rss") or path.startswith("/weather"):
        return False
    if path.startswith("/topic/") or path.startswith("/tag/") or path.startswith("/category/"):
        return False
    if path == "/":
        return False

    # CNA article URLs usually have an ID at the end e.g. ...-5422501
    if not any(ch.isdigit() for ch in path):
        return False

    return True


ISO_DT_RE = re.compile(r"^\d{4}-\d{2}-\d{2}")


def parse_publish_datetime_from_article(html: str) -> datetime | None:
    """
    Try to get publish datetime from:
    1) JSON-LD
    2) Meta tags
    3) <time> tags

    Returns timezone-aware datetime in SGT, or None if not found.
    """
    soup = BeautifulSoup(html, "html.parser")

    # 1) JSON-LD blocks
    for script in soup.find_all("script", type="application/ld+json"):
        try:
            data = json.loads(script.string or "")
        except Exception:
            continue

        candidates = data if isinstance(data, list) else [data]
        for obj in candidates:
            if not isinstance(obj, dict):
                continue
            typ = obj.get("@type") or obj.get("@context")
            if (isinstance(typ, str) and "Article" in typ) or obj.get("headline") or obj.get("datePublished"):
                dt = obj.get("datePublished") or obj.get("dateModified")
                if isinstance(dt, str) and ISO_DT_RE.match(dt):
                    try:
                        dt_parsed = datetime.fromisoformat(dt.replace("Z", "+00:00"))
                        SGT = timezone(timedelta(hours=8))
                        return dt_parsed.astimezone(SGT)
                    except Exception:
                        pass

    # 2) Meta tags
    meta_names = [
        ("meta", {"property": "article:published_time"}),
        ("meta", {"name": "pubdate"}),
        ("meta", {"name": "publish-date"}),
        ("meta", {"name": "date"}),
        ("meta", {"itemprop": "datePublished"}),
        ("meta", {"property": "og:pubdate"}),
        ("meta", {"name": "parsely-pub-date"}),
        ("meta", {"name": "cXenseParse:recs:publishtime"}),
    ]
    for tag, attrs in meta_names:
        el = soup.find(tag, attrs=attrs)
        if el and el.get("content"):
            dt = el["content"].strip()
            if ISO_DT_RE.match(dt):
                try:
                    dt_parsed = datetime.fromisoformat(dt.replace("Z", "+00:00"))
                    SGT = timezone(timedelta(hours=8))
                    return dt_parsed.astimezone(SGT)
                except Exception:
                    pass

    # 3) <time> tags
    for t in soup.find_all("time"):
        dt = (t.get("datetime") or t.get("content") or "").strip()
        if ISO_DT_RE.match(dt):
            try:
                dt_parsed = datetime.fromisoformat(dt.replace("Z", "+00:00"))
                SGT = timezone(timedelta(hours=8))
                return dt_parsed.astimezone(SGT)
            except Exception:
                continue

    return None


def fetch(url: str) -> str | None:
    try:
        r = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT)
        r.encoding = "utf-8"
        r.raise_for_status()
        return r.text
    except Exception as e:
        print(f"[WARN] GET failed: {url} | {e}")
        return None


def within_range(dt: datetime) -> bool:
    """
    Compare by date only against START_DATE / END_DATE.
    dt is assumed to be SGT already.
    """
    d = dt.date()
    return START_DATE <= d <= END_DATE


def date_is_before_start(dt: datetime) -> bool:
    return dt.date() < START_DATE


def extract_listing_links(topic_html: str) -> list[str]:
    """
    Extract candidate article URLs from a CNA topic page.

    CNA topic pages often store the article paths in a `data-link` attribute,
    e.g. data-link="/singapore/ura-new-private-home-prices-sales-up-in-q3-2025-hdb-resale-5422501"
    """
    soup = BeautifulSoup(topic_html, "html.parser")
    hrefs = set()

    # 1) Preferred: grab from data-link attributes
    for el in soup.find_all(attrs={"data-link": True}):
        data_link = el.get("data-link")
        if not data_link:
            continue
        if data_link.startswith("#"):
            continue
        abs_url = urljoin(TOPIC_URL, data_link)
        if is_article_url(abs_url):
            hrefs.add(abs_url)

    # 2) Fallback: <a href="..."> if needed
    if not hrefs:
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if href.startswith("#"):
                continue
            abs_url = urljoin(TOPIC_URL, href)
            if is_article_url(abs_url):
                hrefs.add(abs_url)

    return sorted(hrefs)


# --------------------------
# MAIN
# --------------------------
def main():
    print(f"Scraping CNA Housing articles from {START_DATE} to {END_DATE} ...")
    rows = []
    seen_urls = set()
    reached_older_than_start = False

    for page in range(0, MAX_PAGES):
        page_url = f"{TOPIC_URL}?page={page}"
        print(f"\n[Page {page}] {page_url}")
        html = fetch(page_url)
        if not html:
            print("No HTML returned; stopping.")
            break

        links = extract_listing_links(html)
        if not links:
            print("No candidate links found on page; stopping.")
            break

        print(f"Found {len(links)} candidate links; fetching articles...")
        page_old_count = 0
        page_in_range = 0

        for article_url in links:
            if article_url in seen_urls:
                continue
            seen_urls.add(article_url)

            time.sleep(SLEEP_BETWEEN_REQUESTS)
            article_html = fetch(article_url)
            if not article_html:
                print(f"  [SKIP] No HTML for {article_url}")
                continue

            dt = parse_publish_datetime_from_article(article_html)
            if not dt:
                print(f"  [NO DATE] {article_url}")
                continue

            # quick debug line – comment out later if noisy
            print(f"  [DATE] {dt.date()} | {article_url}")

            if within_range(dt):
                soup = BeautifulSoup(article_html, "html.parser")

                # Title
                meta_title = soup.find("meta", property="og:title")
                if meta_title and meta_title.get("content"):
                    title = meta_title["content"].strip()
                else:
                    t = soup.find("title")
                    title = t.text.strip() if t else ""

                rows.append({
                    "title": title,
                    "url": article_url,
                    "published_utc": dt.isoformat(),
                })
                page_in_range += 1
            elif date_is_before_start(dt):
                page_old_count += 1

        print(f"Kept in range on this page: {page_in_range} | Older-than-start on this page: {page_old_count}")

        if page_in_range == 0 and page_old_count >= 10:
            reached_older_than_start = True
            print("Reached mostly items older than start date; stopping pagination.")
            break

    # Deduplicate by URL and sort by date
    dedup = {r["url"]: r for r in rows}
    final_rows = sorted(dedup.values(), key=lambda r: r["published_utc"])

    fieldnames = ["title", "url", "published_utc"]
    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(final_rows)

    print(f"\nDone. Saved {len(final_rows)} rows to {OUTPUT_CSV}")
    if not reached_older_than_start:
        print("Note: You may increase MAX_PAGES if you think there are more pages within range.")


if __name__ == "__main__":
    main()

import csv
import json
import os
import re
import time
from datetime import datetime, timedelta, timezone
from urllib.parse import urlparse

import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}
REQUEST_TIMEOUT = 20
SLEEP_BETWEEN_REQUESTS = 0.6  # seconds, be polite

SGT = timezone(timedelta(hours=8))
ISO_DT_RE = re.compile(r"^\d{4}-\d{2}-\d{2}")

# Phrase that signals start of housekeeping/ads; stop collecting at this
STOP_MARKER = "sign up for our newsletters"

# Boilerplate lines to drop
_BOILERPLATE_PATTERNS = [
    r"Get CNA updates on WhatsApp",
    r"Subscribe to our newsletter",
    r"Related:?$",
    r"Recommended(?: for you)?:?$",
    r"READ:?$",
    r"READ ALSO:?$",
    r"Follow us on",
    r"Read more:?$",
    r"Get the CNA app",
    r"Get WhatsApp alerts",
    r"Also worth reading",
]
_BOILERPLATE_RE = re.compile("|".join(_BOILERPLATE_PATTERNS), re.IGNORECASE)

# Heuristic content class hints
_CONTENT_CLASS_RE = re.compile(
    r"(article|story|post).*(content|body)|"
    r"(content|text|rich)(-|_)?(body|area|container)",
    re.IGNORECASE
)


# --------------------------
# HELPERS
# --------------------------
def ensure_dirs(path: str):
    d = os.path.dirname(path)
    if d:
        os.makedirs(d, exist_ok=True)


def fetch(url: str) -> str | None:
    try:
        r = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT)
        # Ensure correct decoding to prevent â€œ/â€™ garbling
        r.encoding = "utf-8"
        r.raise_for_status()
        return r.text
    except Exception as e:
        print(f"[WARN] GET failed: {url} | {e}")
        return None


def _clean_text(s: str) -> str:
    s = s.strip()
    s = re.sub(r"\xa0", " ", s)
    s = re.sub(r"[ \t]+", " ", s)
    if _BOILERPLATE_RE.search(s):
        return ""
    return s


def _maybe_fix_misencoded(text: str) -> str:
    """
    If viewer later shows â€œ/â€™ etc it’s usually Excel-view issue,
    but if source is double-decoded, this can help.
    """
    if "â€" in text or "â€™" in text or "â€“" in text:
        try:
            return text.encode("latin1").decode("utf-8")
        except Exception:
            return text
    return text


def _truncate_after_marker(text: str) -> str:
    """Cut content at the stop marker phrase (case-insensitive), if present."""
    if not text:
        return text
    idx = text.lower().find(STOP_MARKER)
    return text[:idx].rstrip() if idx != -1 else text


def parse_publish_datetime_sgt(html: str) -> datetime | None:
    """Parse publish datetime and return timezone-aware datetime in SGT (UTC+8)."""
    soup = BeautifulSoup(html, "html.parser")

    # 1) JSON-LD (prefer datePublished)
    for script in soup.find_all("script", type="application/ld+json"):
        try:
            data = json.loads(script.string or "")
        except Exception:
            continue
        candidates = data if isinstance(data, list) else [data]
        for obj in candidates:
            if not isinstance(obj, dict):
                continue
            typ = obj.get("@type") or obj.get("@context")
            if (isinstance(typ, str) and "Article" in typ) or obj.get("headline") or obj.get("datePublished"):
                dt = obj.get("datePublished") or obj.get("dateModified")
                if isinstance(dt, str) and ISO_DT_RE.match(dt):
                    try:
                        dt_parsed = datetime.fromisoformat(dt.replace("Z", "+00:00"))
                        return dt_parsed.astimezone(SGT)
                    except Exception:
                        pass

    # 2) Meta tags
    meta_names = [
        ("meta", {"property": "article:published_time"}),
        ("meta", {"name": "pubdate"}),
        ("meta", {"name": "publish-date"}),
        ("meta", {"name": "date"}),
        ("meta", {"itemprop": "datePublished"}),
        ("meta", {"property": "og:pubdate"}),
        ("meta", {"name": "parsely-pub-date"}),
        ("meta", {"name": "cXenseParse:recs:publishtime"}),
    ]
    for tag, attrs in meta_names:
        el = soup.find(tag, attrs=attrs)
        if el and el.get("content"):
            dt = el["content"].strip()
            if ISO_DT_RE.match(dt):
                try:
                    dt_parsed = datetime.fromisoformat(dt.replace("Z", "+00:00"))
                    return dt_parsed.astimezone(SGT)
                except Exception:
                    pass

    # 3) <time> tags
    for t in soup.find_all("time"):
        dt = (t.get("datetime") or t.get("content") or "").strip()
        if ISO_DT_RE.match(dt):
            try:
                dt_parsed = datetime.fromisoformat(dt.replace("Z", "+00:00"))
                return dt_parsed.astimezone(SGT)
            except Exception:
                continue

    return None


def extract_article_content(html: str) -> str:
    """Return cleaned plaintext of the article body, truncated at STOP_MARKER."""
    soup = BeautifulSoup(html, "html.parser")

    # 1) JSON-LD articleBody
    try:
        for script in soup.find_all("script", type="application/ld+json"):
            data = json.loads(script.string or "")
            blocks = data if isinstance(data, list) else [data]
            for obj in blocks:
                if isinstance(obj, dict) and obj.get("@type") and "Article" in str(obj.get("@type")):
                    body = obj.get("articleBody")
                    if isinstance(body, str) and len(body.strip()) > 40:
                        text = "\n\n".join(
                            _clean_text(x) for x in re.split(r"\n{2,}", body.strip()) if _clean_text(x)
                        )
                        if text:
                            text = _maybe_fix_misencoded(text)
                            return _truncate_after_marker(text)
    except Exception:
        pass

    def collect_paragraphs(container):
        paras = []
        for p in container.find_all(["p", "h2", "h3"], recursive=True):
            if p.find_parent(["figure", "figcaption", "aside", "blockquote"]):
                continue
            raw = "".join(t for t in p.stripped_strings if isinstance(t, str))
            # Stop if we hit the marker phrase in this paragraph
            if STOP_MARKER in raw.lower():
                break
            txt = _clean_text(raw)
            if txt:
                paras.append(txt)
        return paras

    # 2) <article> block
    article_tag = soup.find("article")
    if article_tag:
        for c in article_tag.find_all(attrs={"itemprop": "articleBody"}):
            paras = collect_paragraphs(c)
            if len(" ".join(paras)) > 100:
                return _truncate_after_marker(_maybe_fix_misencoded("\n\n".join(paras)))
        paras = collect_paragraphs(article_tag)
        if len(" ".join(paras)) > 100:
            return _truncate_after_marker(_maybe_fix_misencoded("\n\n".join(paras)))

    # 3) Heuristic containers
    for div in soup.find_all(["div", "section"], class_=True):
        classes = " ".join(div.get("class") or [])
        if _CONTENT_CLASS_RE.search(classes):
            paras = collect_paragraphs(div)
            if len(" ".join(paras)) > 100:
                return _truncate_after_marker(_maybe_fix_misencoded("\n\n".join(paras)))

    # 4) Last resort: main/body
    main_like = soup.find("main") or soup.body
    if main_like:
        paras = collect_paragraphs(main_like)
        if len(" ".join(paras)) > 100:
            return _truncate_after_marker(_maybe_fix_misencoded("\n\n".join(paras)))

    return ""


def parse_title(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")
    meta_title = soup.find("meta", property="og:title")
    if meta_title and meta_title.get("content"):
        return meta_title["content"].strip()
    t = soup.find("title")
    return t.text.strip() if t else ""


def is_article_url(url: str) -> bool:
    u = urlparse(url)
    if u.netloc not in {"www.channelnewsasia.com", "channelnewsasia.com"}:
        return False
    path = u.path.lower()
    if path.startswith("/watch") or "/watch/" in path:
        return False
    if "/podcast" in path or "/podcasts" in path or path.startswith("/listen"):
        return False
    if path.startswith("/profile") or path.startswith("/rss") or path.startswith("/weather"):
        return False
    if path == "/":
        return False
    return True


def scrape_article(url: str) -> dict:
    html = fetch(url)
    if not html:
        raise RuntimeError("Failed to fetch article HTML.")
    title = parse_title(html)
    published_sgt = parse_publish_datetime_sgt(html)
    content_text = extract_article_content(html)
    return {
        "title": title,
        "url": url,
        "published_sgt_iso": published_sgt.isoformat() if published_sgt else "",
        "content_text": content_text,
        "content_chars": len(content_text) if content_text else 0,
    }


def read_urls_from_csv(path: str) -> list[str]:
    urls = []
    with open(path, "r", encoding="utf-8-sig", newline="") as f:
        reader = csv.reader(f)
        rows = list(reader)
        if not rows:
            return urls
        header = [h.strip().lower() for h in rows[0]]
        data_rows = rows[1:] if any(header) else rows  # headerless CSV
        url_idx = 0
        if any(header):
            if "url" in header:
                url_idx = header.index("url")
            else:
                # Fall back to first column if 'url' not found
                url_idx = 0
        for r in data_rows:
            if not r:
                continue
            u = r[url_idx].strip()
            if u:
                urls.append(u)
    return urls


def save_rows_to_excel(rows: list[dict], output_path: str):
    ensure_dirs(output_path)
    wb = Workbook()
    ws = wb.active
    ws.title = "Articles"

    # Header (added 'content_chars')
    headers = ["title", "url", "published_sgt_iso", "content_text", "content_chars"]
    ws.append(headers)

    # Rows
    for r in rows:
        ws.append([r.get(h, "") for h in headers])

    # Optional: auto width
    for column in ws.columns:
        max_length = 0
        col_letter = column[0].column_letter
        for cell in column:
            try:
                val = "" if cell.value is None else str(cell.value)
                if len(val) > max_length:
                    max_length = len(val)
            except Exception:
                pass
        ws.column_dimensions[col_letter].width = min(max_length + 2, 80)

    wb.save(output_path)
    print(f"✅ Saved Excel file to: {output_path}")


def main():
    print(f"Reading URLs from: {INPUT_CSV}")
    urls = read_urls_from_csv(INPUT_CSV)
    print(f"Found {len(urls)} URL(s).")

    rows = []
    failures = []

    for i, url in enumerate(urls, 1):
        if not is_article_url(url):
            print(f"[{i}/{len(urls)}] Skipping non-article URL: {url}")
            continue

        print(f"[{i}/{len(urls)}] Scraping: {url}")
        time.sleep(SLEEP_BETWEEN_REQUESTS)
        try:
            row = scrape_article(url)
            rows.append(row)
        except Exception as e:
            print(f"[ERROR] {url} | {e}")
            failures.append((url, str(e)))

    save_rows_to_excel(rows, OUTPUT_XLSX)

    if failures:
        print("\nSome URLs failed to scrape:")
        for u, err in failures:
            print(f"- {u} | {err}")

    print(f"\nDone. Scraped {len(rows)} article(s).")


if __name__ == "__main__":
    main()


Scraping CNA Housing articles from 2020-01-01 to 2025-10-31 ...

[Page 0] https://www.channelnewsasia.com/topic/hdb-flat?page=0
Found 11 candidate links; fetching articles...
  [DATE] 2025-04-12 | https://www.channelnewsasia.com/singapore/bto-new-flats-multi-agency-committee-support-residents-5061846
  [DATE] 2025-02-03 | https://www.channelnewsasia.com/singapore/hdb-green-towns-programme-cool-coatings-paint-all-estates-urban-heat-4912396
  [DATE] 2025-04-06 | https://www.channelnewsasia.com/singapore/hdb-neighbourhoods-across-singapore-be-upgraded-mnd-nrp-sup-lup-5047151
  [DATE] 2024-10-25 | https://www.channelnewsasia.com/singapore/hdb-sbf-flats-resale-prices-transactions-rental-4701671
  [DATE] 2025-07-29 | https://www.channelnewsasia.com/singapore/i-saw-my-house-burning-owner-toa-payoh-flat-fire-worries-over-future-5265171
  [DATE] 2025-01-22 | https://www.channelnewsasia.com/singapore/new-initiative-tackle-hoarding-behaviour-singapore-4888641
  [DATE] 2024-10-10 | https://www.cha

In [6]:
# remove 'hdb-flat' from the original scraped file
import pandas as pd
df = pd.read_excel("Output/CNA/cna_articles_2020_Jan_to_2025_Oct.xlsx")
df['website'] = (
    df['website']
    .str.split('|')
    .apply(lambda tags: '|'.join([t for t in tags if t != 'hdb-flat']))
)


In [9]:
# prepare the additional scraped file to merge to original scraped data
hdbflat = pd.read_excel("Output/CNA/cna_hdb_flat.xlsx")
hdbflat.rename(columns={'published_sgt_iso': 'published_date', 'content_text': 'content'}, inplace=True)
hdbflat['website'] = 'hdb-flat'


In [17]:
# merge to original scraped data
df_final = pd.concat([df, hdbflat], ignore_index=True)

In [18]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986 entries, 0 to 985
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   website         986 non-null    object
 1   title           986 non-null    object
 2   url             986 non-null    object
 3   published_date  986 non-null    object
 4   content         986 non-null    object
 5   content_chars   986 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 46.3+ KB


In [27]:
# --- function to merge website tag values in order, no duplicates ---
def merge_tags(series):
    tags = []
    for value in series:
        for t in str(value).split('|'):
            t = t.strip()
            if t and t not in tags:   # keep first occurrence only
                tags.append(t)
    return '|'.join(tags)

# --- group by columns and merge website tag values ---
df_merged = (
    df_final.groupby(['title', 'url', 'published_date', 'content', 'content_chars'], as_index=False)
      .agg({'website': merge_tags})
)


In [28]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           977 non-null    object
 1   url             977 non-null    object
 2   published_date  977 non-null    object
 3   content         977 non-null    object
 4   content_chars   977 non-null    int64 
 5   website         977 non-null    object
dtypes: int64(1), object(5)
memory usage: 45.9+ KB


In [30]:
df_merged.to_excel('Output/CNA/final_cna_articles_2020_Jan_to_2025_Oct.xlsx', index=False)

# clean the scraped data

In [195]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

In [196]:
df = pd.read_excel("Output/CNA/final_cna_articles_2020_Jan_to_2025_Oct.xlsx")

In [197]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      977 non-null    int64 
 1   title           977 non-null    object
 2   url             977 non-null    object
 3   published_date  977 non-null    object
 4   content         977 non-null    object
 5   content_chars   977 non-null    int64 
 6   website         977 non-null    object
dtypes: int64(2), object(5)
memory usage: 53.6+ KB


In [198]:
# drop unwanted columns
df = df.drop(columns=['Unnamed: 0', 'website'])

In [199]:
# check if any duplicates
df['title'].duplicated().any()


False

In [200]:
# check if any duplicates
df['url'].duplicated().any()


False

In [201]:
pd.set_option('display.max_colwidth', None)
print(df.loc[df['title'] == "A couple’s stylish 1,000 sq ft BTO flat with a DIY whisky bar and teak wood touches", 'content'].to_string(index=False))

CNA Lifestyle\n\nCNA Lifestyle’s Making Room series looks at small homes with big transformations. This week, see how a “build-it-yourself” approach resulted in a stylish home with luxurious marble, teak and brass finishes.\n\nHomeowners Jasvin Kaur and Jasdep Singh wanted their 1,000 sq ft four-room HDB BTO flat in Telok Blangah to showcase a few things: A contemporary-meets-luxe aesthetic, hubby Jasdep’s burgeoning whisky collection, and their Indian heritage.\n\nAnd their ingenious DIY tweaks with some teak wood plays a role in realising all three.\n\nIt wasn’t easy, given their different style preferences. “I always liked luxury. Jasdep likes things more contemporary,” explained Jasvin, an assistant manager in a company.\n\nBut “this was a chance to express ourselves with what we have always wanted,” added Jasdep, who is a general manager at Originals, a furniture store that specialises in unique and authentic home decor pieces sourced from all over the world.\n\n“The fact that I w

In [202]:
# remove irrelevant phrases
import re

# ensure column is string
df['content'] = df['content'].astype(str)

# remove the ABOUT AUTHOR phrases
df['content'] = df['content'].str.replace(r'ABOUT THE AUTHOR:.*', '', regex=True).str.strip()

# remove the ABOUT AUTHORS phrases
df['content'] = df['content'].str.replace(r'ABOUT THE AUTHORS:.*', '', regex=True).str.strip()

# remove READ: ... lines
df['content'] = df['content'].str.replace(
    r'READ:\s.*?(?=\n\n|$)',   # match READ: ... until next \n\n
    '',
    regex=True
).str.strip()

# remove READ> ... lines
df['content'] = df['content'].str.replace(
    r'READ>\s.*?(?=\n\n|$)',   # match READ: ... until next \n\n
    '',
    regex=True
).str.strip()

# remove WATCH> ... lines
df['content'] = df['content'].str.replace(
    r'WATCH>\s.*?(?=\n\n|$)',   # match READ: ... until next \n\n
    '',
    regex=True
).str.strip()

# other lines to remove
lines_to_remove = [
    "This audio is generated by an AI tool.",
    "This article was originally published inTODAY.",
    "For more news like this, visittodayonline.com.",
    "A single handpicked story that we think you shouldn't miss. Just one a day.",
    "\n\nRelated Topics",
    "Week in Review\n\nSubscribe to our Chief Editor’s Week in Review\n\nOur chief editor shares analysis and picks of the week's biggest news every Saturday.",
    "Living\n\nCNA Lifestyle’s Making Room series looks at small homes with big transformations.",
    "CNA Lifestyle\n\nCNA Lifestyle’s Making Room series looks at small homes that were given big transformations.",
    "CNA Lifestyle\n\nCNA Lifestyle’s Making Room series looks at small homes with big transformations."
    "CNA Insider\n\nSubscribe to the best of CNA Insider\n\nCNA's best current affairs documentaries with a deeper look at issues affecting Asia, sent to your inbox weekly",
    "Watch thisTalking Point special here. The programme airs on Channel 5 every Thursday at 9.30pm.",    
    "Morning Brief\n\nSubscribe to CNA’s Morning Brief\n\nAn automated curation of our top stories to start your day.",
    "Have views on this issue or a news topic you care about? Send your letter tovoices[at]mediacorp.com.sgwith your full name, address and phone number.",
    "CNA Womenis a section on CNA Lifestyle that seeks to inform, empower and inspire the modern woman. If you have women-related news, issues and ideas to share with us, emailCNAWomen[at]mediacorp.com.sg.",
    "Subscribe to CNA's Recommended ReadThis service is not intended for persons residing in the E.U. By clicking subscribe, I agree to receive news updates and promotional material from Mediacorp and Mediacorp’s partners."
]

pattern = "|".join(re.escape(line) for line in lines_to_remove)

df['content'] = (
    df['content']
    .str.replace(pattern, "", regex=True)
    #.str.replace(r"\s+", " ", regex=True)  # clean extra spaces
    .str.strip()
)

In [203]:
# extract first line
df['first_lines'] = df['content'].str.split('\n\n', n=1).str[0]

# drop rows which are not of Singapore context based on first line
df = df[~df['first_lines'].isin(['Asia', 'East Asia', 'Malaysia', 'World'])]


In [204]:
# parse 'content' column and check if there are any Singapore-related keywords

# Singapore-related keywords
keywords = [
    "Singapore", "Singapore's", "Singaporean", "Singaporeans",
    "HDB BTO", "BTO flat", "HDB flat", "BTO HDB", "HDB home",
    "Bukit Panjang", "resale flat",
    "Housing and Development Board",
    "Hougang",
    "Good Class Bungalows", "Good Class Bungalow",
    "Selective En bloc Redevelopment Scheme"
]

# Build regex pattern
pattern = r"\b(" + "|".join(re.escape(k) for k in keywords) + r")\b"

# Combine title + content into one string to search
df["Singapore_context"] = (
    (df["title"].str.contains(pattern, case=False, regex=True)) |
    (df["content"].str.contains(pattern, case=False, regex=True))
).map({True: "yes", False: "no"})

# drop rows which are not of Singapore context
df = df[df['Singapore_context'] != 'no']


In [205]:
len(df)

912

In [206]:
# further cleaning of 'content' column
# clean the first lines of 'content' column
patterns = [
    r"^Adulting\b",
    r"^Advertorial\b",
    r"^Brand Spotlight\b",
    r"^Business\b",
    r"^CNA Insider\b",
    r"^CNA Lifestyle\b",
    r"^Commentary\b",
    r"commentaryCommentary",
    r"^Entertainment\b",
    r"^Ground Up\b",
    r"^Living\b",
    r"^Obsessions\b",
    r"^People\b",
    r"^Remarkable Living\b",
    r"^Trending\b",
    r"^Up Close\b",
    r"^Voices\b",
    r"^Women\b",
    r"^Singapore\b",
    r"^SINGAPORE:\b"
]

# Combine patterns into one regex
pattern = r"(" + "|".join(patterns) + r")"

df["content"] = df["content"].str.replace(pattern, "", regex=True).str.strip()

# one more round of cleaning up SINGAPORE: and SINGAPORE -
# lines that should be removed ONLY if they appear at the start of the text
lines_to_remove = [
    "SINGAPORE:",
    "SINGAPORE —",
    "CNA Lifestyle’s Making Room series looks at small homes with big transformations.",
    "CNA Lifestyle’s Making Room series returns to look at small homes with big transformations.",
    "CNA Lifestyle’s Making Room series look at small homes with big transformations.",
    "CNA Lifestyle’s new series Making Room takes a look at small homes that have been given big transformations."    
]

# Build regex so each item is anchored at the start with ^ 
pattern = "|".join(rf"^{re.escape(line)}" for line in lines_to_remove)

df["content"] = (
    df["content"]
    .str.replace(pattern, "", regex=True)
    .str.strip()
)


In [207]:
# keep only relevant columns
df = df[['title', 'url', 'published_date', 'content', 'content_chars']]

In [208]:
df.to_excel("Output/CNA/cleaned_cna_articles_2020_Jan_to_2025_Oct.xlsx", index=False)
